In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
from data import trainGenerator, testGenerator, saveResult, testGenerator2
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans

ModuleNotFoundError: No module named 'data'

In [ ]:
# Code to crop images to specific size

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def crop_image(src, save_path):
    TEST_SET = os.listdir(src)
    img_h = 256
    img_w = 256
    stride = img_h-40
    for n in range(len(TEST_SET)):
        image_name = TEST_SET[n]
        path1 = image_name[0:-7]+'mask.png'  #rename mask
        # load the image
        # image = cv2.imread(os.path.join(src,image_name), cv2.IMREAD_UNCHANGED)
        # image = cv2.imread(os.path.join(src,image_name))
        print(os.path.join(src,image_name))
        image = io.imread(os.path.join(src,image_name))
        
        print(image.shape)
        h, w, _ = image.shape
        # h, w = image.shape

        num = 0;
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], croping:{}'.format(n+1, len(TEST_SET), image_name))

        #mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w] 
                if num <= 9:
                    #path1 = image_name[0:-4]+'0'+ str(num)+'.jpg'
                    path1 = image_name[0:-4]+'0'+ str(num)+'.png'
                else:
                    #path1 = image_name[0:-4]+str(num)+'.jpg'
                    path1 = image_name[0:-4]+str(num)+'.png'
                
                #io.imsave(save_path + path1, crop)
                cv2.imwrite(save_path + path1, crop)
                num = num + 1
        print('Done!')

In [ ]:
import os
import shutil

# Directory containing the PNG images
source_directory = '/kaggle/input/deepglobe-road-extraction-dataset/train'

# Directory where PNG images will be moved
satelliteDir = "/kaggle/working/satellite_images/"
maskedDir = "/kaggle/working/masked_images/" 

# Create the subdirectory if it doesn't exist
if not os.path.exists(satelliteDir):
    os.makedirs(satelliteDir)
    
if not os.path.exists(maskedDir):
    os.makedirs(maskedDir)

# List all files in the directory
all_files = os.listdir(source_directory)

# Filter and move PNG files to the subdirectory
for file in all_files:
    if file.endswith('.png'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(maskedDir, file)
        
        # Move the file to the subdirectory
        shutil.move(source_path, destination_path)
    elif file.endswith('.jpg'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(satelliteDir, file)
        
        # Move the file to the subdirectory
        shutil.move(source_path, destination_path)

In [ ]:
# Cropping Train Images

cropsatDir = "/kaggle/working/cropped_satellite_images/"
cropmaskDir = "/kaggle/working/cropped_masked_images/" 

if not os.path.exists(cropsatDir):
    os.makedirs(cropsatDir)
    
if not os.path.exists(cropmaskDir):
    os.makedirs(cropmaskDir)

crop_image(satelliteDir, cropsatDir)
crop_image(maskedDir, cropmaskDir)

In [ ]:
# Cropping Test images - To Be Done

# satelliteDir = "/kaggle/working/cropped_satellite_images/"

# directory = '/kaggle/input/deepglobe-road-extraction-dataset/test'
# all_files = os.listdir(directory)

# # Filter only PNG files
# maskedImgs = [file for file in all_files if file.endswith('.png')]
# satelImgs = [file for file in all_files if file.endswith('.jpg')]
        
# crop_image(satelImgs, satelliteDir)
# crop_image(maskedImgs, maskedDir)

In [1]:
model = unet_model(input_shape=(572, 572, 3), num_classes=2)

NameError: name 'unet_model' is not defined

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=0, mode='min', epsilon=1e-4, 
                              cooldown=0, min_lr=1e-6)
visual = TensorBoard(log_dir='./D_resunet1_log', histogram_freq=0, write_graph=True, write_images=True)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')

train_Gene = trainGenerator(2,'/kaggle/working/','cropped_satellite_images/','cropped_masked_images/',data_gen_args,save_to_dir = None)
# to be fixed(train-val split)
# val_Gene = trainGenerator(2,'data/membrane/IEEE_road/test/','sat_img_crops','masks_crops',data_gen_args)

In [ ]:
model_checkpoint = ModelCheckpoint('U-Net.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model.fit(train_Gene,steps_per_epoch=50,epochs=10,
                    callbacks=[model_checkpoint, visual, reduce_lr, earlystop], 
                    validation_data=val_Gene, validation_steps=220)#step_per_epoch and validation_steps equals to number of samples divide batchsize

In [ ]:
fileDir = "data/membrane/IEEE_road/test/test_images/"  #test images(1024*1024)
#fileDir = "data/membrane/train/f"
preDir = "data/membrane/IEEE_road/test/sub_test/predict/" #Dir of predict mask

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def predict_z(src, predict_path):
    TEST_SET = os.listdir(src)
    model = D_resunet()
    #model = res_unet1()
    print('Loading Model weights...')
    model.load_weights('D_resunet.hdf5')
    print('completed!')
    img_h = 256
    img_w = 256
    stride = img_h-16
    for n in range(len(TEST_SET)):
        path = TEST_SET[n]
        path1 = path[0:-7]+'mask.png'  #rename mask
        # load the image
        image = io.imread(os.path.join(src,path))
        h, w, _ = image.shape

        image = image / 255.0
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], predicting:{}'.format(n+1, len(TEST_SET), path))

        mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w, :3] #[****)
                
                ch, cw, _ = crop.shape

                if ch != img_h or cw != img_h:
                    print('invalid size!')
                    print(i, j, h_begin, w_begin, ch, cw)
                    break
                
                crop = np.expand_dims(crop, axis=0)
                pred = model.predict(crop, verbose=2)
                pred = pred.reshape((img_h, img_h, 1)).astype(np.float64)
                #pred = np.argmax(pred, axis=2)
                #print(pred.shape)
                #pred = np.array(pred)
                
                pred[pred >= 0.5] = 1
                pred[pred < 0.5] = 0
                pred = pred * 255
                '''
                for a in range(img_h):
                    for b in range(img_h):
                        if pred[a, b] == 0.:
                            temp[a, b, :] = [223, 223, 223]
                        elif pred[a, b] == 1.:
                            temp[a, b, :] = [255, 204, 163]
                        else:
                            print('Unknown type:', pred[a, b])
                '''
                mask_whole[h_begin:h_begin + img_h, w_begin:w_begin + img_w] \
                    = pred
                # + mask_whole[i * stride:i * stride + image_size, j * stride:j * stride + image_size, :]
        cv2.imwrite(predict_path + path1, mask_whole[0:h, 0:w])
        #print('Done!')
        
predict_z(fileDir, preDir)  

In [ ]:
def encoder_block(inputs, num_filters): 
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3,padding = 'valid')(inputs) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Max Pooling with 2x2 filter 
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2)(x) 
    return x

In [ ]:
def decoder_block(inputs, skip_features, num_filters): 
    # Upsampling with 2x2 filter 
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides = 2, padding = 'valid')(inputs) 
      
    # Copy and crop the skip features  
    # to match the shape of the upsampled input 
    skip_features = tf.image.resize(skip_features, size = (x.shape[1], x.shape[2])) 
    x = tf.keras.layers.Concatenate()([x, skip_features]) 
      
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
  
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
      
    return x

In [ ]:
# Code for Road Extraction using 
# Vanilla U-Net Architecture
import tensorflow as tf 
  
def unet_model(input_shape = (256, 256, 3), num_classes = 1): 
    inputs = tf.keras.layers.Input(input_shape) 
      
    # Contracting Path 
    s1 = encoder_block(inputs, 64) 
    s2 = encoder_block(s1, 128) 
    s3 = encoder_block(s2, 256) 
    s4 = encoder_block(s3, 512) 
      
    # Bottleneck 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(s4) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(b1) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
      
    # Expansive Path 
    s5 = decoder_block(b1, s4, 512) 
    s6 = decoder_block(s5, s3, 256) 
    s7 = decoder_block(s6, s2, 128) 
    s8 = decoder_block(s7, s1, 64) 
      
    # Output 
    outputs = tf.keras.layers.Conv2D(num_classes, 1, padding = 'valid', activation = 'sigmoid')(s8) 
      
    model = tf.keras.models.Model(inputs = inputs, outputs = outputs, name = 'U-Net') 
    return model 